In [ ]:
import numpy as np
from smt.surrogate_models import KRG, RBF
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
import keras
from keras import layers
import keras_tuner


In [ ]:
df = pd.read_csv('Monte_Carlo_10000/training_data_bending.csv')

In [ ]:
X = df[['outer_wall_thickness', 'inside_wall_side_thickness', 'inside_wall_middle_thickness', 'height', 'width', 'sigma0', 'youngs']].values
Y = df[['mean_force']].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=(X_train.shape[1],)))
    #model.add(layers.Dropout(rate=0.2))
    #activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    units = hp.Int('units', min_value=16, max_value=128, step=16)
    num_layers = hp.Int('num_layers', 1, 10)
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    for i in range(num_layers):
        model.add(layers.Dense(
            units=units, 
            activation='relu',
            ))
        #model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(
        units=1, 
        activation='relu'))
    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate),
        loss='mean_squared_error',
        metrics=['mae']
        )
    return model

hp = keras_tuner.HyperParameters()
build_model(hp)

tuner = keras_tuner.GridSearch(
    hypermodel=build_model,
    objective='val_mae',
    executions_per_trial=5,
    directory='keras_tuner',
    project_name='test_15'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(
    X_train, 
    Y_train, 
    epochs=1000,
    batch_size=32,
    validation_split=0.2, 
    callbacks=[early_stopping]
)



In [ ]:
# Retrieve the hyperparameters and performance
results = []
for trial in tuner.oracle.get_best_trials(num_trials=1000):
    result = {
        'units': trial.hyperparameters.get(f'units'),
        'num_layers': trial.hyperparameters.get('num_layers'),
        'learning_rate': trial.hyperparameters.get('learning_rate'),
        'mse': trial.score
    }
    results.append(result)

# Create DataFrame
hyperparameters_df = pd.DataFrame(results)

In [ ]:
models = tuner.get_best_models(num_models=5)
best_model = models[0]

In [ ]:
tuner.results_summary()

In [ ]:
hyperparameters_df.head()
#hyperparameters_df.to_csv('hyperparameters_1.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterMathtext

# Load the hyperparameters DataFrame
#hyperparameters_1_df = pd.read_csv('hyperparameters_1.csv')
#hyperparameters_2_df = pd.read_csv('hyperparameters_2.csv')
#combined_df = pd.concat([hyperparameters_1_df, hyperparameters_2_df])

# Set learning rates to create one plot per learning rate
learning_rates = hyperparameters_df['learning_rate'].unique()
learning_rates = np.sort(learning_rates)
print(learning_rates)

# Determine the global minimum and maximum MSE values
min_mse = hyperparameters_df['mse'].min()
max_mse = hyperparameters_df['mse'].max()

# Adjust general plot settings
plt.rc('font', size=22)

# Loop through each learning rate and create a separate figure
for lr in learning_rates:
    # Filter the dataset for each learning rate
    subset = hyperparameters_df[hyperparameters_df['learning_rate'] == lr]
    
    # Create a new figure
    plt.figure(figsize=(8, 6), dpi=300)
    scatter = plt.scatter(
        x=subset['units'], 
        y=subset['num_layers'], 
        c=subset['mse'],  # Use the original MSE values
        cmap='coolwarm', 
        norm=LogNorm(vmin=min_mse, vmax=max_mse),  # Apply consistent logarithmic normalization
        s=500, 
        edgecolor='none',  # Dots without edges
        marker='o'  # Use dots as markers
    )

    # Set up color bar with scientific notation
    colorbar = plt.colorbar(scatter, label='Validation MAE')
    formatter = LogFormatterMathtext(base=10)  # Use scientific notation for ticks
    colorbar.ax.yaxis.set_major_formatter(formatter)

    plt.xlabel('Units')
    plt.ylabel('Number of Layers')
    
    # Save or show each plot separately without a title
    plt.tight_layout()
    plt.show()
